# Initialization

In [ ]:
!spark-shell --version

In [ ]:
!pip install --quiet minio

# Titanic Logistic Regression using PySpark
This is a code along of the famous titanic dataset, its always nice to start off with this dataset because it is an example you will find across pretty much every data analysis language.

# Download data from public

In [ ]:
!wget https://raw.githubusercontent.com/meddash-cloud/meddash-public-datasets/main/data/walmart_stock.csv

# Upload the CSV file to Minio

In [ ]:
BUCKET_NAME="spark-stream-batching-on-minio-csvs"
minio_url = "minio-service.kubeflow.svc.cluster.local:9000"
minio_key = "minio"
minio_secret = "minio123"

In [ ]:
csv_file_name = "walmart_stock.csv"

In [ ]:
from minio import Minio
from minio.error import S3Error
import os

upload_file_name=csv_file_name
upload_file_path="./{}".format(upload_file_name)
config = {
    "endpoint": minio_url,
    "access_key": "minio",
    "secret_key": "minio123",
    "secure": False,
    }


# Create a client with the MinIO server playground, its access key
# and secret key.
print ("connecting to minio {}".format(minio_url))
minio_client = Minio(**config)

print("try to find bucket {}".format(BUCKET_NAME))
found = minio_client.bucket_exists(BUCKET_NAME)
print("found", found)
if not found:
    minio_client.make_bucket(BUCKET_NAME)
else:
    print("Bucket '{}' already exists".format(BUCKET_NAME))

print("upload file to minio...")
minio_client.fput_object(BUCKET_NAME, os.path.basename(upload_file_path), upload_file_path)

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
minio_file_path="s3a://{}".format(BUCKET_NAME)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import os

cwd = os.getcwd()

minio_url = "minio-service.kubeflow.svc.cluster.local:9000"
spark = SparkSession\
.builder\
.appName("ReadTextFilesFromS3")\
.master("local[*]")\
.config('spark.jars.packages','org.apache.hadoop:hadoop-aws:3.3.4')\
.config("spark.hadoop.fs.s3a.endpoint", "http://"+minio_url)\
.config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
.config("spark.hadoop.fs.s3a.path.style.access", "true")\
.config("spark.hadoop.fs.s3a.access.key", "minio")\
.config("spark.hadoop.fs.s3a.secret.key", "minio123")\
.getOrCreate()



sc = spark.sparkContext


